# Stickleback case study data prep
CSV -> Parquet

For details about Parquet and Python, see https://arrow.apache.org/docs/python/parquet.html.

Before proceeding, get the case study data from [here](https://drive.google.com/drive/folders/1OHZNjPtekwC8cQ0YviPSz0GNl6Q86kNh). Download the v4 folder as a zip file, rename it "sticklebackdatav4.zip" and put it in the same folder as this notebook. Yes, this is a hacky solution. No, I don't have time to make it more robust.

In [1]:
import os
import pandas as pd
import shutil
import zipfile

## Read data
Unzip data and load CSV files into memory as Pandas DataFrames.

In [2]:
# Check the v4 data exists 
if not os.path.exists("sticklebackdatav4.zip"):
    raise FileNotFoundError("sticklebackdatav4.zip not found")

# Unzip v4 data
if os.path.exists("v4"):
    shutil.rmtree("v4")
os.makedirs("v4")
with zipfile.ZipFile("sticklebackdatav4.zip", "r") as v4zip:
    v4zip.extractall("v4")

In [3]:
# Read CSVs
sensors = pd.read_csv("v4/sensors.csv").set_index("datetime")
sensors.index = pd.to_datetime(sensors.index)
events = pd.read_csv("v4/events.csv").set_index("datetime")
events.index = pd.to_datetime(events.index)

In [20]:
foo = sensors.groupby("deployid").agg(lambda df: (df.index.max() - df.index.min()).total_seconds() / 3600).sort_values("depth")
# foo["depth"].median()
#events.groupby(["deployid", "event"]).size().groupby(level="event").agg(["median", "min", "max"])
foo

,depth,pitch,roll,jerk,speed
deployid,,,,,
bw180830-52a,0.771556,0.771556,0.771556,0.771556,0.771556
bw180904-52,1.556806,1.556806,1.556806,1.556806,1.556806
bw170815-21,2.381528,2.381528,2.381528,2.381528,2.381528
bw180830-48,2.640972,2.640972,2.640972,2.640972,2.640972
bw180827-53,2.865250,2.865250,2.865250,2.865250,2.865250
bw180830-52b,2.947111,2.947111,2.947111,2.947111,2.947111
bw180829-30,4.235750,4.235750,4.235750,4.235750,4.235750
bw170816-27,4.368361,4.368361,4.368361,4.368361,4.368361
bw170814-50,7.737528,7.737528,7.737528,7.737528,7.737528


In [12]:
(sensors.index.max() - sensors.index.min()).total_seconds() / 3600

9342.757638888888

## Convert to Parquet
Put the output in a directory called "v5". Make sure the "pyarrow" package is installed.

In [ ]:
if os.path.exists("v5"):
    shutil.rmtree("v5")
os.makedirs("v5")
sensors.to_parquet("v5/sensors.parquet")
events.to_parquet("v5/events.parquet")